Importing all the useful libraries...

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Checking the version of TensorFlow...

In [2]:
!pip show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: gast, numpy, keras-preprocessing, wrapt, keras-applications, tensorflow-estimator, astor, six, protobuf, termcolor, wheel, tensorboard, absl-py, google-pasta, grpcio
Required-by: stable-baselines, magenta, fancyimpute


Need to import data...

In [3]:
!pwd

/content
